In [2]:
#Ryan Shin: sungjin7127@gmail.com
#Date: 160927
#Obj: find the best model for emotion recogntion

from __future__ import print_function
import numpy as np
import tensorflow as tf
import sys
import math
from six.moves import cPickle as pickle
from six.moves import range

import os
#%matplotlib inline
print ("Packages loaded")

#Load data
cwd = os.getcwd()
loadpath = cwd + "/dataset/data_gray.npz"
l = np.load(loadpath)

#Check what's included
print (l.files)

# Parse data
trainimg = l['trainimg']
trainlabel = l['trainlabel']
testimg = l['testimg']
testlabel = l['testlabel']
imgsize = l['imgsize']
#use_gray = l['use_gray']
ntrain = trainimg.shape[0]
nclass = trainlabel.shape[1]
dim    = trainimg.shape[1]
ntest  = testimg.shape[0]
print ("%d train images loaded" % (ntrain))
print ("%d test images loaded" % (ntest))
print ("%d dimensional input" % (dim))
print ("Image size is %s" % (imgsize))
print ("%d classes" % (nclass))

Packages loaded
['trainlabel', 'trainimg', 'imgsize', 'testimg', 'testlabel']
2614 train images loaded
1743 test images loaded
4096 dimensional input
Image size is [64 64]
8 classes



In [4]:
print ('Training set', trainimg.shape, trainlabel.shape)
#print 'Validation set', valid_dataset.shape, valid_labels.shape
print ('Test set', testimg.shape, testlabel.shape)

Training set (2614, 4096) (2614, 8)
Test set (1743, 4096) (1743, 8)


In [8]:
image_size = 64
num_labels = 8
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(trainimg, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print ('Training set', train_dataset.shape, train_labels.shape)
#print 'Validation set', valid_dataset.shape, valid_labels.shape
print ('Test set', test_dataset.shape, test_labels.shape)

NameError: name 'train_labels' is not defined

In [4]:
datasets = {
    "image_size": 64,
    "label_count": 8,
    "channel_count": 1
}
datasets["total_image_size"] = datasets["image_size"] * datasets["image_size"]

{'channel_count': 1,
 'image_size': 64,
 'label_count': 8,
 'total_image_size': 4096}

In [8]:
trainimg.shape
trainlabel.shape

(2614, 8)

In [11]:
def reformat(dataset, labels, name):
    dataset = dataset.reshape((-1, datasets["image_size"], datasets["image_size"], datasets["channel_count"])).astype(np.float32)
    # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(datasets["label_count"]) == labels[:,None]).astype(np.float32)
    print(name + " set", dataset.shape, labels.shape)
    return dataset, labels
datasets["train"], datasets["train_labels"] = reformat(trainimg, trainlabel, "Training")
#datasets["valid"], datasets["valid_labels"] = reformat(valid_dataset, valid_labels, "Validation")
datasets["test"], datasets["test_labels"] = reformat(testimg, testlabel, "Test")

print(datasets.keys())

Training set (2614, 64, 64, 1) (2614, 1, 8)
Test set (1743, 64, 64, 1) (1743, 1, 8)
['total_image_size', 'train_labels', 'test_labels', 'channel_count', 'label_count', 'train', 'image_size', 'test']


In [12]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [13]:
def run_graph(graph_info, data, step_count, report_every=50):
    with tf.Session(graph=graph_info["graph"]) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        batch_size = graph_info["batch_size"]
        for step in xrange(step_count + 1):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (data["train_labels"].shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = data["train"][offset:(offset + batch_size), :, :, :]
            batch_labels = data["train_labels"][offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            targets = [graph_info["optimizer"], graph_info["loss"], graph_info["predictions"]]
            feed_dict = {graph_info["train"] : batch_data, graph_info["labels"] : batch_labels}
            _, l, predictions = session.run(targets, feed_dict=feed_dict)
            if (step % report_every == 0):
                print("Minibatch loss at step", step, ":", l)
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                #print("Validation accuracy: %.1f%%" % accuracy(graph_info["valid"].eval(), data["valid_labels"]))
        print("Test accuracy: %.1f%%" % accuracy(graph_info["test"].eval(), data["test_labels"]))

In [14]:
def convnet_two_layer(batch_size, patch_size, depth, hidden_size, data):
    image_size = data["image_size"]
    label_count = data["label_count"]
    channel_count = data["channel_count"]
    graph = tf.Graph()
    with graph.as_default():
        # Input data.
        train = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, channel_count))
        labels= tf.placeholder(tf.float32, shape=(batch_size, label_count))
        #valid = tf.constant(data["valid"])
        test  = tf.constant(data["test"])

        # Variables.
        layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, channel_count, depth], stddev=0.1))
        layer1_biases  = tf.Variable(tf.zeros([depth]))
        layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
        layer2_biases  = tf.Variable(tf.constant(1.0, shape=[depth]))
        layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, hidden_size], stddev=0.1))
        layer3_biases  = tf.Variable(tf.constant(1.0, shape=[hidden_size]))
        layer4_weights = tf.Variable(tf.truncated_normal([hidden_size, label_count], stddev=0.1))
        layer4_biases  = tf.Variable(tf.constant(1.0, shape=[label_count]))

          # Model.
        def model(set):
            conv   = tf.nn.conv2d(set, layer1_weights, [1, 2, 2, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer1_biases)
            conv   = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer2_biases)
            shape  = hidden.get_shape().as_list()
            reshape= tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
            hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
            return tf.matmul(hidden, layer4_weights) + layer4_biases

        # Training computation.
        logits = model(train)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels))
        
        info = {
            "graph": graph,
            "batch_size": batch_size,
            "train": train,
            "labels": labels,
            "loss": loss,
            "optimizer": tf.train.GradientDescentOptimizer(0.05).minimize(loss),

            # Predictions for the training, validation, and test data.
            "predictions": tf.nn.softmax(logits),
            #"valid": tf.nn.softmax(model(valid)),
            "test":  tf.nn.softmax(model(test))
        }
    return info

In [ ]:
graph_2conv = convnet_two_layer(batch_size=16, patch_size=5, depth=16, hidden_size=64, data=datasets)

run_graph(graph_2conv, datasets, 1000)

In [ ]:
#Ryan Shin: sungjin7127@gmail.com
#Date: 160927

import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
print ("Packages loaded")

#Load data
cwd = os.getcwd()
loadpath = cwd + "/dataset/data_gray.npz"
l = np.load(loadpath)

#Check what's included
print (l.files)

#Parse data
trainimg = l['trainimg']
trainlabel = l['trainlabel']
testimg = l['testimg']
testlabel = l['testlabel']
ntrain = trainimg.shape[0]
nclass = trainlabel.shape[1]
dim = trainimg.shape[1]
ntest = testimg.shape[0]
print ("%d train images loaded" % (ntrain))
print ("%d test images loaded" % (ntest))
print ("%d dimentional input" % (dim))
print ("%d classes" % (nclass))

#Define network
tf.set_random_seed(0)
#Param. of Log. Regression
learning_rate = 0.001
training_epochs = 1000
batch_size = 10
display_step = 100


graph = tf.Graph()

with graph.as_default():

    #Saver init
    ckpt_dir = './ckpt_dir'
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    global_step = tf.Variable(0, name='global_step', trainable=False)

    # Create Graph for Logistic Regression
    x = tf.placeholder("float", [None, dim])
    y = tf.placeholder("float", [None, nclass])
    W = tf.Variable(tf.zeros([dim, nclass]), name = 'weights')
    b = tf.Variable(tf.zeros([nclass]))

    #Define functions
    WEIGHT_DECAY_FACTOR = 0.000001
    l2_loss = tf.add_n([tf.nn.l2_loss(v)
                       for v in tf.trainable_variables()])
    _pred = tf.nn.softmax(tf.matmul(x, W) + b)
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(_pred),
                                        reduction_indices=1))
    cost = cost + WEIGHT_DECAY_FACTOR*l2_loss
    #Optimizer
    optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    _corr = tf.equal(tf.argmax(_pred, 1), tf.argmax(y, 1))
    accr = tf.reduce_mean(tf.cast(_corr, tf.float32))
    saver = tf.train.Saver()
    print ("Functions ready")

with tf.Session(graph=graph) as session:
    init = tf.initialize_all_variables().run()
    ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(session, ckpt.model_checkpoint_path) #restore all Variable
    start = global_step.eval() # get last globak

    print("global_step :", start)
    #Launch the graph
    #sess = tf.Session()
    #Optimize
    #sess.run(init)
    #Training Cycle
    for epoch in xrange(training_epochs):
        avg_cost = 0.
        num_batch = int(ntrain/batch_size)
        # Loop over all batches
        for i in xrange(num_batch):
            randidx = np.random.randint(ntrain, size=batch_size)
            batch_xs = trainimg[randidx, :]
            batch_ys = trainlabel[randidx, :]
            # Fit Training using batch data
            session.run(optm, feed_dict={x: batch_xs, y: batch_ys})
            # Compute Average loss
            avg_cost += session.run(cost,
                                feed_dict = {x: batch_xs, y: batch_ys})/num_batch

            if (epoch % 10 == 0):
                #print('Minbath loss at step ', epoch, ':', avg_cost)
                global_step.assign(epoch).eval()
                saver.save(session, ckpt_dir + '/model.ckpt', global_step=global_step)

            # Display logs per epoch step

            if epoch % display_step == 0:
                train_acc = session.run(accr, feed_dict={x: batch_xs, y: batch_ys})
                print ("Epoch: %03d/%03d cost: %.9f" %
                      (epoch, training_epochs, avg_cost))
                print (" Training accuracy: %.3f" % (train_acc))
                test_acc = session.run(accr, feed_dict={x: testimg, y: testlabel})
                print (" Test accuracy: %.3f" % (test_acc))


print ("Optimization Finished")
#sess.close()
print ("Session closed.")


In [ ]:
tf.set_random_seed(0)
n_input = dim
n_output = nclass
weights = {
    'wc1' : tf.Variable(tf.random_normal([5,5,1,128], stddev=0.1)),
    'wc2' : tf.Variable(tf.random_normal([5,5,128,128], stddev=0.1)),
    'wd1' : tf.Variable(tf.random_normal(
            [(int)(imgsize[0]/4*imgsize[1]/4)*128, 128], stddev=0.1)),
    'wd2' : tf.Variable(tf.random_normal([128, n_output], stddev=0.1))
}

biases = {
    'bc1' : tf.Variable(tf.random_normal([128], stddev=0.1)),
    'bc2' : tf.Variable(tf.random_normal([128], stddev=0.1)),
    'bd1' : tf.Variable(tf.random_normal([128], stddev=0.1)),
    'bd2' : tf.Variable(tf.random_normal([n_output], stddev=0.1)),
}

def conv_basic(_input, _w, _b, _keepratio):
    # INPUT
    _input_r = tf.reshape(_input, shape=[-1, imgsize[0], imgsize[1], 1])
    # CONVOLUTION LAYER 1
    _conv1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(_input_r
        , _w['wc1'], strides=[1, 1, 1, 1], padding='SAME'), _b['bc1']))
    _pool1 = tf.nn.max_pool(_conv1, ksize=[1, 2, 2, 1]
        , strides=[1, 2, 2, 1], padding='SAME')
    _pool_dr1 = tf.nn.dropout(_pool1, _keepratio)
    # CONVOLUTION LAYER 2
    _conv2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(_pool_dr1
        , _w['wc2'], strides=[1, 1, 1, 1], padding='SAME'), _b['bc2']))
    _pool2 = tf.nn.max_pool(_conv2, ksize=[1, 2, 2, 1]
        , strides=[1, 2, 2, 1], padding='SAME')
    _pool_dr2 = tf.nn.dropout(_pool2, _keepratio)
    # VECTORIZE
    _dense1 = tf.reshape(_pool_dr2
                         , [-1, _w['wd1'].get_shape().as_list()[0]])
    # FULLY CONNECTED LAYER 1
    _fc1 = tf.nn.relu(tf.add(tf.matmul(_dense1, _w['wd1']), _b['bd1']))
    _fc_dr1 = tf.nn.dropout(_fc1, _keepratio)
    # FULLY CONNECTED LAYER 2
    _out = tf.add(tf.matmul(_fc_dr1, _w['wd2']), _b['bd2'])
    # RETURN
    out = {
        'input_r': _input_r, 'conv1': _conv1, 'pool1': _pool1
        , 'pool1_dr1': _pool_dr1, 'conv2': _conv2, 'pool2': _pool2
        , 'pool_dr2': _pool_dr2, 'dense1': _dense1, 'fc1': _fc1
        , 'fc_dr1': _fc_dr1, 'out': _out
    }
    return out
print ("NETWORK READY")

#Define function

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])
keepratio = tf.placeholder(tf.float32)

# Functions!
_pred = conv_basic(x, weights, biases, keepratio)['out']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(_pred, y))
WEIGHT_DECAY_FACTOR = 0.0001
l2_loss = tf.add_n([tf.nn.l2_loss(v)
            for v in tf.trainable_variables()])
cost = cost + WEIGHT_DECAY_FACTOR*l2_loss
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
_corr = tf.equal(tf.argmax(_pred,1), tf.argmax(y,1)) # Count corrects
accr = tf.reduce_mean(tf.cast(_corr, tf.float32)) # Accuracy
init = tf.initialize_all_variables()
print ("FUNCTIONS READY")

# Parameters
training_epochs = 400
batch_size      = 100
display_step    = 40

# Launch the graph
sess = tf.Session()
sess.run(init)

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(ntrain/batch_size)+1
    # Loop over all batches
    for i in range(num_batch):
        randidx = np.random.randint(ntrain, size=batch_size)
        batch_xs = trainimg[randidx, :]
        batch_ys = trainlabel[randidx, :]
        # Fit training using batch data
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys
                                  , keepratio:0.7})
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys
                                , keepratio:1.})/num_batch

    # Display logs per epoch step
    if epoch % display_step == 0 or epoch == training_epochs-1:
        print ("Epoch: %03d/%03d cost: %.9f" %
               (epoch, training_epochs, avg_cost))
        train_acc = sess.run(accr, feed_dict={x: batch_xs
                                , y: batch_ys, keepratio:1.})
        print (" Training accuracy: %.3f" % (train_acc))
        test_acc = sess.run(accr, feed_dict={x: testimg
                                , y: testlabel, keepratio:1.})
        print (" Test accuracy: %.3f" % (test_acc))
print ("Optimization Finished!")

sess.close()
print ("Session closed.")


NETWORK READY
FUNCTIONS READY
